In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [ ]:
df = pd.read_csv("clean_scout.csv")

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.head(5).T

In [ ]:
df.isnull().sum()*100 / df.shape[0]

In [ ]:
miss_val = []

[miss_val.append(column) for column in df.columns if any(df[column].isnull())]

print("Columns that have missing values :")

miss_val

In [ ]:
# function for first looking to the columns

def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print("--------------------------------")
    print(df[col].value_counts(dropna = False))

In [ ]:
def fill_most_freq(df, group_col, col_name):
    
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill_prop(df, group_col, col_name):
    
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
first_looking("age")

In [ ]:
df['age'].fillna('-', inplace=True)

In [ ]:
df["age"].value_counts(dropna=False)

In [ ]:
df.groupby("age").km.describe()

In [ ]:
df[df["age"] == "-"]["km"].value_counts(dropna=False)

In [ ]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

In [ ]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

In [ ]:
df.groupby('age').km.mean()

In [ ]:
df["age"].value_counts(dropna=False)

In [ ]:
df.groupby(['make_model',"body_type", 'age']).price.describe()

In [ ]:
df['age'].replace('-', 0, inplace=True)

In [ ]:
df.groupby('age').km.mean()

In [ ]:
df["age"].value_counts(dropna=False)

In [ ]:
first_looking("km")

In [ ]:
df.groupby("age").km.mean()

In [ ]:
df[df["km"].isna()][["age", "km"]]

In [ ]:
df.groupby("age").km.transform("mean")

In [ ]:
df[df["km"].isna()][["age", "km"]]

In [ ]:
df.km.value_counts(dropna=False)

In [ ]:
first_looking("body_type")

In [ ]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [ ]:
df['body_type'].value_counts(dropna=False)

In [ ]:
df["body_type"].mode()

In [ ]:
df["body_type"].mode()[0]

In [ ]:
for group in df["make_model"].unique():
    cond = df["make_model"]==group
    mode = df[cond]["body_type"].mode().to_list()  # list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [ ]:
df['body_type'].value_counts(dropna=False)

In [ ]:
def fill_most_freq(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_most_freq(df, "make_model", "body_type")

In [ ]:
first_looking("Previous_Owners")

In [ ]:
df["Previous_Owners"].fillna("-", inplace = True)

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
df.groupby(['make_model', 'body_type', 'age', 'Previous_Owners']).price.describe()

In [ ]:
df.groupby(['make_model', 'body_type', 'age', 'Previous_Owners']).km.describe()

In [ ]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")
df.loc[cond, "Previous_Owners"] = 0.0

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [ ]:
for age in df["age"].unique():
    cond = df["age"] == age
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")  # If there has still been "NaN" values this code line will deal with it

In [ ]:
df["Previous_Owners"].value_counts(dropna=False)

In [ ]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    
    for group in df[group_col].unique():
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_prop(df, "age", "Previous_Owners")

In [ ]:
first_looking("Warranty")

In [ ]:
df["Warranty"].fillna("-", inplace = True)

In [ ]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

In [ ]:
df.groupby(["make_model", "Warranty"]).price.describe().sort_values(by = ["make_model", "mean"], ascending=False)

In [ ]:
df.drop("Warranty", axis=1, inplace=True)

In [ ]:
first_looking("vat")

In [ ]:
df["vat"].fillna("-", inplace = True)

In [ ]:
df.groupby(["make_model", "body_type", "vat"]).price.describe()

In [ ]:
df.drop("vat", axis=1, inplace=True)

In [ ]:
first_looking("Paint_Type")

In [ ]:
df["Paint_Type"].fillna("-", inplace = True)

In [ ]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

In [ ]:
df.groupby(["make_model", "body_type", 'Paint_Type']).price.describe().sort_values(by = ["make_model", "body_type", "mean"], ascending = False)

In [ ]:
df["Paint_Type"].replace("-", np.nan, inplace=True)

In [ ]:
for group1 in df["make_model"].unique():
    for group2 in list(df["body_type"].unique()):
        cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
        df.loc[cond2, "Paint_Type"] = df.loc[cond2, "Paint_Type"].fillna(method="ffill").fillna(method="bfill")
                
for group1 in list(df["make_model"].unique()):
    cond1 = df["make_model"]==group1
    df.loc[cond1, "Paint_Type"] = df.loc[cond1, "Paint_Type"].fillna(method="ffill").fillna(method="bfill")            
           
df["Paint_Type"] = df["Paint_Type"].fillna(method="ffill").fillna(method="bfill")

In [ ]:
df["Paint_Type"].value_counts(dropna=False)

In [ ]:
def fill_ffill(df, group_col1, group_col2, col_name):
    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
    for group1 in list(df[group_col1].unique()):
        cond1 = df[group_col1]==group1
        df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_ffill(df, "make_model", "body_type", "Paint_Type")

In [ ]:
first_looking("Type")

In [ ]:
df.Type.fillna("-", inplace=True)

In [ ]:
df.groupby(["Type", "make_model", "age"]).km.describe()

In [ ]:
df.groupby(["make_model", "age", "Type"]).price.describe()

In [ ]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [ ]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-', 'New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-', 'Used')

In [ ]:
df['Type'].value_counts(dropna=False)

In [ ]:
first_looking("Inspection_new")

In [ ]:
df["Inspection_new"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

In [ ]:
df["Inspection_new"].replace("-", "No", inplace=True)

In [ ]:
df["Inspection_new"].value_counts(dropna=False)

In [ ]:
df["Inspection_new"].replace(["Yes", "No"], [1, 0], inplace=True)

In [ ]:
df["Inspection_new"].value_counts(dropna=False)

In [ ]:
first_looking("Upholstery_type")

In [ ]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe()

In [ ]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], 
                              ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [ ]:
df["Upholstery_type"].value_counts(dropna=False)

In [ ]:
df["Upholstery_type"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe()

In [ ]:
df["Upholstery_type"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

In [ ]:
df.drop("Upholstery_color", axis=1, inplace=True)

In [ ]:
first_looking("Nr_of_Doors")

In [ ]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        cond1 = df["make_model"]==group1
        cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
        mode1 = list(df[cond1]["Nr_of_Doors"].mode())
        mode2 = list(df[cond2]["Nr_of_Doors"].mode())
        if mode2 != []:
            df.loc[cond2, "Nr_of_Doors"] = df.loc[cond2, "Nr_of_Doors"].fillna(df[cond2]["Nr_of_Doors"].mode()[0])
        elif mode1 != []:
            df.loc[cond2, "Nr_of_Doors"] = df.loc[cond2, "Nr_of_Doors"].fillna(df[cond1]["Nr_of_Doors"].mode()[0])
        else:
            df.loc[cond2, "Nr_of_Doors"] = df.loc[cond2, "Nr_of_Doors"].fillna(df["Nr_of_Doors"].mode()[0])

In [ ]:
df["Nr_of_Doors"].value_counts(dropna=False)

In [ ]:
def fill_mode(df, group_col1, group_col2, col_name):
    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond1 = df[group_col1]==group1
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            mode1 = list(df[cond1][col_name].mode())
            mode2 = list(df[cond2][col_name].mode())
            if mode2 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
            elif mode1 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
            else:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
    
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
fill_mode(df, "make_model", "body_type", "Nr_of_Doors")

In [ ]:
first_looking("Nr_of_Seats")

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

In [ ]:
first_looking("Cylinders")

In [ ]:
df["Cylinders"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "age", "Cylinders"]).price.describe()

In [ ]:
df.drop("Cylinders", axis=1, inplace=True)

In [ ]:
first_looking("Drive_chain")

In [ ]:
df["Drive_chain"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

In [ ]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [ ]:
df.loc[cond, 'Drive_chain'] = df.loc[cond, 'Drive_chain'].replace('-', '4WD')

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

In [ ]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

In [ ]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

In [ ]:
first_looking("Emission_Class")

In [ ]:
df["Emission_Class"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

In [ ]:
df.drop("Emission_Class", axis=1, inplace=True)

In [ ]:
first_looking("Gears")

In [ ]:
df["Gears"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

In [ ]:
df["Gears"].replace("-", np.nan, inplace=True)

In [ ]:
for group1 in df["make_model"].unique():
    for group2 in df["body_type"].unique():
        for group3 in df["Gearing_Type"].unique():
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [ ]:
df["Gears"].value_counts(dropna=False)

In [ ]:
first_looking("hp_kW")

In [ ]:
df["hp_kW"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

In [ ]:
df["hp_kW"].replace(["-"], np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

In [ ]:
first_looking("Displacement_cc")

In [ ]:
df["Displacement_cc"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

In [ ]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

In [ ]:
first_looking("Weight_kg")

In [ ]:
df["Weight_kg"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

In [ ]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

In [ ]:
first_looking("CO2_Emission")

In [ ]:
df["CO2_Emission"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Fuel", "CO2_Emission"]).price.describe()

In [ ]:
df.groupby(["Fuel", "make_model", "body_type", "CO2_Emission"]).price.describe().loc["Electric"]

In [ ]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [ ]:
df.loc[df.Fuel == "Electric", "CO2_Emission"]

In [ ]:
df.loc[df.Fuel == "Electric", "CO2_Emission"] = df.loc[df.Fuel == "Electric", "CO2_Emission"].fillna(df.loc[df.Fuel == "Electric", "CO2_Emission"].mode()[0])
df.loc[df.Fuel == "Electric", "CO2_Emission"]

In [ ]:
df['CO2_Emission'].fillna(df.groupby(["make_model", "body_type", "Fuel"])["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df.groupby(["make_model", "body_type"])["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df.groupby("make_model")["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df["CO2_Emission"].median(), inplace = True)

In [ ]:
df["CO2_Emission"].value_counts(dropna=False)

In [ ]:
def fill_median(df, group_col1, group_col2, group_col3, col_name):
    
    df[col_name].fillna(df.groupby([group_col1, group_col2, group_col3])[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df[col_name].median(), inplace = True)

    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))        

In [ ]:
fill_median(df, "make_model", "body_type", "Fuel", "CO2_Emission")

In [ ]:
first_looking("Comfort_Convenience")

In [ ]:
fill(df, "make_model", "body_type", "Comfort_Convenience", "mode")

In [ ]:
first_looking("Entertainment_Media")

In [ ]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

In [ ]:
first_looking("Extras")

In [ ]:
fill(df, "make_model", "body_type", "Extras", "mode")

In [ ]:
first_looking("Safety_Security")

In [ ]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

In [ ]:
first_looking("cons_comb")

In [ ]:
cons_comb_2 = (df["cons_country"] + df["cons_city"])/2
cons_comb_2.value_counts(dropna=False)

In [ ]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb_2)

In [ ]:
df["cons_comb"].value_counts(dropna=False)

In [ ]:
df["cons_comb"].fillna("-", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Fuel", "cons_comb"]).price.describe()

In [ ]:
df.groupby(["Fuel", "make_model", "body_type", "cons_comb"]).price.describe().loc["Electric"]

In [ ]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [ ]:
df.loc[df.Fuel == "Electric", "cons_comb"]

In [ ]:
df.loc[df.Fuel == "Electric", "cons_comb"] = df.loc[df.Fuel == "Electric", "cons_comb"].fillna(df.loc[df.Fuel == "Electric", "cons_comb"].mode()[0])
df.loc[df.Fuel == "Electric", "cons_comb"]

In [ ]:
df["cons_comb"].value_counts(dropna=False)

In [ ]:
fill_median(df, "make_model", "body_type", "Fuel", "cons_comb")

In [ ]:
df.drop("cons_country", axis = 1, inplace = True)

In [ ]:
df.drop("cons_city", axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()*100 / df.shape[0]

In [ ]:
df.to_csv("filled_scout.csv", index=False)